<a href="https://colab.research.google.com/github/poring3995/Assignment-9-/blob/main/Collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark
     

#pip install spark
     

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("/content/spark-3.0.0-bin-hadoop3.2/data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=10, regParam=0.5, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

Root-mean-square error = 1.2274912663121302


**Conclusion :** 

for the model after I do some experimental with with several hyperparameters values (maxIter and regParam) as I try it with 4 cases model

1st model is using maxIter = 5. regParam = 0.1

Average RMSE (based on running for 10 times) = 1.1736360154494294

2nd model is using maxIter = 10, regParam = 0.1

Average RMSE (based on running for 10 times) = 1.0568265521064983

3rd model is using maxIter = 5, regParam = 0.5

Average RMSE (based on running for 10 times) = 1.2015760364620547

4th model is using maxIter = 10, regParam = 0.5

Average RMSE (based on running for 10 times) = 1.1851709836369329


So, base on the experimental I would say that the 2nd model is the best performance among these 4 models since it has the less RMSE, according to the changing the value of the hyper parameter with base on these 4 models experimental, we can see that the more number of iteration loop would provide more accuracy result which may due to the factor that the model has more step to be improved and prediction, but for the regularizer parameter we can see that it quite opposite to the Iteration loop since the more more value of it would causes the lower accuracy to the model, since it would aim to reduce the overfitting of the model and with too much regularizer rate it could make the model much less effective which could be decrease the model accuracy.
